# HCS Calculation (suite)

The goal of this mini project is to build the entire HCS calculation (based on the PHR.ipynb mini project)

In [3]:
import numpy as np
from sun_phy.mr_ofdm.fields import HCS_calculation

# Constant part (See sun_phy fields.py line 57)
# Generator polynomial x^8 + x^2 + x + 1
G = np.poly1d([1, 0, 0, 0, 0, 0, 1, 1, 1])
# part a)
k = np.poly1d(np.block([1, np.zeros(22)]))
b = np.poly1d(np.ones(8))
c = np.polymul(k, b)
_, A = np.polydiv(c, G)
A = np.mod(A, 2).astype(int)
print(f"A = {A}")

A = [0 1 0 0 1 0 1 1]


In [33]:
def HCS_calculation_expand(input_array):
    """
    Applies HCS crc to input array
    See 18.2.1.3 PHR

    Parameters
    ----------
    input_array : ndarray
        Input values (0 and 1) in a numpy array. [0] is the first element
    
    Returns
    -------
    output_array : ndarray
        Calculated CRC
    """
    # Generator polynomial x^8 + x^2 + x + 1
    G = np.poly1d([1, 0, 0, 0, 0, 0, 1, 1, 1])
    # part a)
    k = np.poly1d(np.block([1, np.zeros(22)]))
    b = np.poly1d(np.ones(8))
    c = np.polymul(k, b)
    _, A = np.polydiv(c, G)
    A = np.mod(A, 2).astype(int)
    # part b)
    a = np.polymul(np.poly1d(input_array), np.poly1d(np.block([1, np.zeros(8)])))
    _, B = np.polydiv(a, G)
    # Pad B to have the same size as A (this has been tested and is identical)
    if B.order + 1 < 8:
        B = np.block([np.zeros(8 - (len(B) + 1)), B])
    B = np.mod(B, 2).astype(int)

    # one's complement of the modulo-2 sum    
    output_array = 1 - np.mod(np.polyadd(A, B), 2)
    return A, B, output_array

In [40]:
# New HCS calculation, synthesizable
def HCS_new(message):
    #A = np.array([0, 1, 0, 0, 1, 0, 1, 1])
    A = 0b01001011

    def B_step(B, inp):
        msb = (B & (1<<7)) >> 7
        return (B << 1 ^ (0b111 * (inp ^ msb))) & 0b1111_1111

    B = 0b0000_0000
    for m in message:
        B = B_step(B, m)

    return np.array([int(x) for x in np.binary_repr(A ^ B ^ 0b1111_1111, 8)])


message = [1,0,0,0,0,0,0,0,0]
hcs_new = HCS_new(message)
hcs = HCS_calculation(message)
print(hcs)
print(hcs_new)

[1 0 1 0 0 0 0 1]
[1 0 1 0 0 0 0 1]


In [43]:
for _ in range(10_000):
    message = np.random.randint(0,2,22)
    hcs_new = HCS_new(message)
    hcs = HCS_calculation(message)

    assert np.array_equal(hcs, hcs_new)